In [ ]:
import pygame
import os
pygame.font.init()
pygame.mixer.init()

WIDTH, HEIGHT = 900,500
WIN = pygame.display.set_mode((WIDTH,HEIGHT))
BLACK=(0,0,0)
RED = (255,0,0)
BLUE = (0,0,255)
WHITE = (255,255,255)
BORDER = pygame.Rect(WIDTH//2 - 5 ,0,10 ,HEIGHT)

BULLET_HIT_SOUND = pygame.mixer.Sound(os.path.join('E:\\pygame\\Assets\\Grenade+1.mp3'))
BULLET_FIRED_SOUND = pygame.mixer.Sound(os.path.join('E:\\pygmae\\Assets\\Gun+Silencer.mp3'))

HEALTH_FONT = pygame.font.SysFont('comicsans',40)
WINNER_FONT = pygame.font.SysFont('comicsans',100)

Player_1_spaceship= pygame.image.load(os.path.join('E:\pygame\Assets\spaceship_yellow.png'))
player1_spaceship= pygame.transform.rotate(pygame.transform.scale(Player_1_spaceship,(65,40)), 270)

Player_2_spaceship= pygame.image.load(os.path.join('E:\pygame\Assets\spaceship_red.png'))
player2_spaceship= pygame.transform.rotate(pygame.transform.scale(Player_2_spaceship,(55,40)) ,90)

FPS = 60
VEL = 5
BULLET_VEL = 8
MAX_Bullets = 4

blue_hit = pygame.USEREVENT + 1
red_hit = pygame.USEREVENT + 2

pygame.display.set_caption(os.path.join('Space war'))

SPACE = pygame.transform.scale(pygame.image.load('E:\pygmae\Assets\space.png'), (WIDTH, HEIGHT))

def draw_window(player1, player2,player1_Bullets,player2_Bullets,red_health,blue_health):
    WIN.blit(SPACE, (0, 0))
    pygame.draw.rect(WIN,BLACK,BORDER)
    
    red_health_text = HEALTH_FONT.render("Health:" + str(red_health),1,WHITE)
    blue_health_text = HEALTH_FONT.render("Health:" + str(blue_health),1,WHITE)
    
    WIN.blit(red_health_text,(WIDTH - red_health_text.get_width() - 10 , 10))
    WIN.blit(blue_health_text,(10 , 10))
    
    WIN.blit(player1_spaceship,(player1.x, player1.y))
    WIN.blit(player2_spaceship,(player2.x, player2.y))

    
    for bullet in player2_Bullets:
        pygame.draw.rect(WIN,RED,bullet)
    
    for bullet in player1_Bullets:
        pygame.draw.rect(WIN,BLUE,bullet)
    
    pygame.display.update() 

def player1_movement(keys_pressed,player1):
     if keys_pressed[pygame.K_a] and  player1.x > 0: #Move Left
            player1.x -= VEL
     if keys_pressed[pygame.K_d] and  player1.x + player1.width < 455 : #Move Right
            player1.x += VEL
     if keys_pressed[pygame.K_w] and player1.y > 0: 
            player1.y -= VEL
     if keys_pressed[pygame.K_s] and player1.y + player1.height < HEIGHT -20:
            player1.y += VEL
            

def player2_movement(keys_pressed,player2):
     if keys_pressed[pygame.K_LEFT] and player2.x - VEL >BORDER.x + BORDER.width: #Move Left
            player2.x -= VEL
     if keys_pressed[pygame.K_RIGHT] and player2.x + player2.width < 920: #Move Right
            player2.x += VEL
     if keys_pressed[pygame.K_UP] and player2.y > 0: 
            player2.y -= VEL
     if keys_pressed[pygame.K_DOWN] and player2.y + player2.height < HEIGHT -20 :
            player2.y += VEL

def handle_bullets(player1_Bullets, player2_Bullets,player1,player2):
    for bullet in player1_Bullets:
        bullet.x += BULLET_VEL
        if player2.colliderect(bullet):
            pygame.event.post(pygame.event.Event(red_hit))
            player1_Bullets.remove(bullet)
        elif bullet.x > WIDTH:
            player1_Bullets.remove(bullet)
            
            
    for bullet in player2_Bullets:
        bullet.x -= BULLET_VEL
        if player1.colliderect(bullet):
            pygame.event.post(pygame.event.Event(blue_hit))
            player2_Bullets.remove(bullet)
        elif bullet.x < 0:
            player2_Bullets.remove(bullet)

def draw_winner(text):
    draw_text = WINNER_FONT.render(text,1,WHITE)
    WIN.blit(draw_text,(WIDTH/2 - draw_text.get_width()/2 , HEIGHT/2 - draw_text.get_height()/2))
    pygame.display.update()
    pygame.time.delay(5000)
    
            
            

def main():
    player1=pygame.Rect(200,175,65,40)
    player2=pygame.Rect(700,175,55,40)
    
    player1_Bullets = []
    player2_Bullets = []
    
    red_health = 10 
    blue_health = 10 
    
    clock = pygame.time.Clock()
    
    
    
    run = True
    while run:
        clock.tick(FPS)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                
                
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LCTRL and len(player1_Bullets) < MAX_Bullets:
                    bullet = pygame.Rect(player1.x + player1.width,player1.y + player1.height//2 + 7  ,10 ,5)
                    player1_Bullets.append(bullet)  
                    BULLET_FIRED_SOUND.play()
                    
                
                if event.key == pygame.K_RCTRL and len(player2_Bullets) < MAX_Bullets :
                    bullet = pygame.Rect(player2.x + player2.width//4,player2.y + player2.height//2 + 7 ,10,5)
                    player2_Bullets.append(bullet)
                    BULLET_FIRED_SOUND.play()
            
            if event.type == red_hit:
                red_health -= 1
                BULLET_HIT_SOUND.play()
            
            

            if event.type == blue_hit:
                blue_health -= 1
                BULLET_HIT_SOUND.play()
                
                
        winner_text = ""        
        if red_health <= 0 :
            winner_text = "Blue Wins!"
            
        if blue_health <= 0 :
            
            winner_text = "Red Wins!"
        
        if winner_text != "":
            draw_winner(winner_text)
            break
                    
        print(player1_Bullets,player2_Bullets)            
        keys_pressed = pygame.key.get_pressed()
        player1_movement(keys_pressed,player1)
        player2_movement(keys_pressed,player2)
        
        
        handle_bullets(player1_Bullets,player2_Bullets,player1,player2)
        draw_window(player1, player2,player1_Bullets,player2_Bullets,red_health,blue_health)


    main()
if __name__=="__main__":
    main()
